# Итоговый проект: разработка гибридной системы рекомендаций книг с нейросетевыми моделями

**Выполнил:** Груданов Николай Алексеевич
**Студнт группы:** М24-525

## Настройка окружения

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split as surprise_train_test_split
from tqdm import tqdm
import itertools
import torch
import torch.cuda.memory as memory
import numpy as np
import os
import sys

In [ ]:
# Импорт библиотек и настройка путей
from pathlib import Path
import os
import numpy as np
import pandas as pd

PROJECT_ROOT = Path.cwd()
DATA_DIR = (PROJECT_ROOT / "data" / "goodbooks-10k").resolve()
print(f"Текущая рабочая директория: {PROJECT_ROOT}")
print(f"Ожидаемая локальная папка с данными: {DATA_DIR}")

In [ ]:
# Проверка локального наличия датасета Goodbooks-10k
REQUIRED_FILES = ["ratings.csv", "books.csv", "tags.csv", "book_tags.csv"]

missing_files = [fname for fname in REQUIRED_FILES if not (DATA_DIR / fname).exists()]
if missing_files:
    raise FileNotFoundError(
        "Не найдены файлы датасета: " + ", ".join(missing_files) + f". Ожидаем путь {DATA_DIR}"
    )
else:
    for fname in REQUIRED_FILES:
        file_path = DATA_DIR / fname
        size_mb = file_path.stat().st_size / (1024 ** 2)
        print(f"Файл {fname} найден ({size_mb:.2f} MB)")

In [ ]:
def check_gpu_environment() -> tuple[str, torch.device, bool]:
    """
    Проверка GPU окружения с красивым форматированным выводом.
    
    Объединяет функциональность проверки доступности CUDA/ROCm,
    определения устройства и вывода детальной информации о GPU.
    
    Returns:
        tuple: (device_str: str, device: torch.device, is_gpu_available: bool)
            - device_str: строковое представление устройства ('cuda' или 'cpu')
            - device: объект torch.device для использования в коде
            - is_gpu_available: флаг доступности GPU (True/False)
    """
    # Импорты (функция самодостаточна)
    import torch
    import torch.cuda.memory as memory
    
    # Разделитель заголовка
    print("\n" + "="*70)
    print("🔍 ПРОВЕРКА GPU ОКРУЖЕНИЯ".center(70))
    print("="*70)
    
    # Базовая информация о PyTorch
    print(f"\n📦 PyTorch версия: {torch.__version__}")
    
    # Проверка доступности CUDA/ROCm
    is_gpu_available = torch.cuda.is_available()
    
    if is_gpu_available:
        # Определение типа GPU (CUDA или ROCm)
        if torch.version.hip:
            gpu_type = "ROCm (AMD)"
            gpu_version = torch.version.hip
            print(f"✅ {gpu_type} обнаружен (версия: {gpu_version})")
        else:
            gpu_type = "CUDA (NVIDIA)"
            gpu_version = torch.version.cuda
            print(f"✅ {gpu_type} обнаружен (версия: {gpu_version})")
        
        # Информация о GPU
        device_count = torch.cuda.device_count()
        print(f"\n📊 Информация о GPU:")
        print(f"   • Количество GPU: {device_count}")
        
        # Детальная информация по каждому GPU
        for i in range(device_count):
            print(f"\n   GPU {i}:")
            print(f"   • Имя: {torch.cuda.get_device_name(i)}")
            total_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"   • Объём VRAM: {total_memory:.2f} GB")
            print(f"   • Вычислительные блоки: {torch.cuda.get_device_properties(i).multi_processor_count}")
        
        # Определение устройства
        device_str = 'cuda'
        device = torch.device('cuda')
        
        # Разделитель
        print("\n" + "="*70)
        print(f"🎯 УСТРОЙСТВО: {device}".center(70))
        print(f"📌 ТИП: {device_str}".center(70))
        print("="*70 + "\n")
        
    else:
        # GPU недоступен
        print(f"⚠️ CUDA/ROCm недоступен")
        print(f"💡 Вычисления будут выполняться на CPU")
        
        # Определение устройства
        device_str = 'cpu'
        device = torch.device('cpu')
        
        # Разделитель
        print("\n" + "="*70)
        print(f"🎯 УСТРОЙСТВО: {device}".center(70))
        print(f"📌 ТИП: {device_str}".center(70))
        print("="*70 + "\n")
    
    return device_str, device, is_gpu_available

In [ ]:
# Пример использования:
# device_str, device, is_gpu_available = check_gpu_environment()
# if is_gpu_available:
#     model = model.to(device)
# else:
#     print("Используем CPU для обучения")
